<a href="https://colab.research.google.com/github/alirezasdg1/Amazon-recommender-system/blob/master/notebooks/NN_TextReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df_all_15 = pd.read_csv("/content/drive/My Drive/Datasets/df_all_15_text.csv")

In [22]:
!wget https://recommender-system-amazon.s3.amazonaws.com/df_content_recommender.csv

--2020-09-16 21:06:22--  https://recommender-system-amazon.s3.amazonaws.com/df_content_recommender.csv
Resolving recommender-system-amazon.s3.amazonaws.com (recommender-system-amazon.s3.amazonaws.com)... 52.216.109.203
Connecting to recommender-system-amazon.s3.amazonaws.com (recommender-system-amazon.s3.amazonaws.com)|52.216.109.203|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4046747265 (3.8G) [binary/octet-stream]
Saving to: ‘df_content_recommender.csv’

df_content_recommen 100%[===================>]   3.77G  35.3MB/s    in 1m 51s  

2020-09-16 21:08:13 (34.9 MB/s) - ‘df_content_recommender.csv’ saved [4046747265/4046747265]



In [24]:
df_content_recommender = pd.read_csv("df_content_recommender.csv")

In [25]:
df_content_recommender.shape

(316350, 3082)

In [20]:
df_all_15

,Unnamed: 0,category,description,title,also_buy,image_x,brand,feature,rank,also_view,main_cat,price,asin,overall,verified,reviewText,summary,unixReviewTime,style,item_text,title_nlp
0,0,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,5.0,False,your interest explor area new york beyond metr...,Great for exploring the rural parts of the state,1470009600,{'Format:': ' Paperback'},garmingreat explor rural part stateformat pape...,delorm new york state atla amp gazett
1,1,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,4.0,True,wonder comprehens atla littl larg,Four Stars,1438646400,{'Format:': ' Paperback'},garminfour starsformat paperback,delorm new york state atla amp gazett
2,2,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,5.0,True,great product fast deliveri need rv travel,Super!,1432771200,{'Format:': ' Paperback'},garminsuperformat paperback,delorm new york state atla amp gazett
3,3,"['Sports & Outdoors', 'Outdoor Recreation', 'C...","[""Rely on delorme ATLAS & gazetteer paper maps...",Garmin DeLorme Atlas &amp; Gazetteer Paper Map...,"['0899334482', '0899334474', '0899333834', '19...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","['>#84,484 in Office Products (See top 100)', ...",[],Office Products,$21.96,0899333257,5.0,True,delorm atla us state theyr amaz detail accur e...,Necessary for off grid camping,1505260800,{'Format:': ' Paperback'},garminnecessari grid campingformat paperback,garmin delorm atla amp gazett paper map arizona
4,4,"['Sports & Outdoors', 'Outdoor Recreation', 'C...","[""Rely on delorme ATLAS & gazetteer paper maps...",Garmin DeLorme Atlas &amp; Gazetteer Paper Map...,"['0899334482', '0899334474', '0899333834', '19...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","['>#84,484 in Office Products (See top 100)', ...",[],Office Products,$21.96,0899333257,5.0,False,first delorm atla gazett ever purchas year ago...,Don't explore without it,1470614400,,garmindont explor without,garmin delorm atla amp gazett paper map arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370247,370255,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Pusdon Instant Reusable Cooling Towels are d...,"Soft Cooling Towel for Instant Relief, 48inch ...","['B07CHSRVDB', 'B07CR8MQRM', 'B00JLRTZRM', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,Pusdon,['GET 1 FREE Silicone Hand Strengthener Grip R...,"54,146 in Sports & Outdoors (","['B00NIYOOMA', 'B07L1TYGZW', 'B01K97XFFK', 'B0...",Sports & Outdoors,$10.99,B01HJDGJ1E,5.0,False,choic type cool towel dri rock hard crispi kin...,"Excellent length, good colors.",1468972800,{'Color:': ' Bamboo Charcoal: 1Gray+1Blue'},pusdonexcel length good colorscolor bamboo cha...,soft cool towel instant relief 

In [4]:
df_all_15 = df_all_15[~(df_all_15['title_nlp'].isna())]
df_all_15 = df_all_15[~(df_all_15['item_text'].isna())]
df_all_15 = df_all_15[~(df_all_15['reviewText'].isna())]

In [5]:
df_items_text = df_all_15.groupby(['asin', 'title','title_nlp','image_x'])['item_text'].apply(lambda x: ' '.join(x)).reset_index()

In [6]:
df_review_text = df_all_15.groupby(['asin', 'title','title_nlp','image_x'])['reviewText'].apply(lambda x: ' '.join(x)).reset_index()

In [7]:
merged_df = df_items_text.merge(df_review_text, how='inner', on=['asin', 'title','title_nlp','image_x'])

In [8]:
merged_df.head(4)

,asin,title,title_nlp,image_x,item_text,reviewText
0,0899332757,Delorme New York State Atlas &amp; Gazetteer,delorm new york state atla amp gazett,['https://images-na.ssl-images-amazon.com/imag...,garmingreat explor rural part stateformat pape...,your interest explor area new york beyond metr...
1,0899333257,Garmin DeLorme Atlas &amp; Gazetteer Paper Map...,garmin delorm atla amp gazett paper map arizona,['https://images-na.ssl-images-amazon.com/imag...,garminnecessari grid campingformat paperback g...,delorm atla us state theyr amaz detail accur e...
2,0971100764,BenchMaster Pocket Guide - Fly Fishing - Fishing,benchmast pocket guid fli fish fish,['https://images-na.ssl-images-amazon.com/imag...,pocket guidesbut make great stock stufferforma...,husband know everyth tie knot make great stock...
3,3843518912,Spooner Boards Freestyle - Green,spooner board freestyl green,['https://images-na.ssl-images-amazon.com/imag...,spooner boardsgreat old spooner boardsa good a...,purchas year old son love play time coldraini ...


In [9]:
merged_df["reviewText"].iloc[:4]

0    your interest explor area new york beyond metr...
1    delorm atla us state theyr amaz detail accur e...
2    husband know everyth tie knot make great stock...
3    purchas year old son love play time coldraini ...
Name: reviewText, dtype: object

In [14]:
review_embeddings = hub.text_embedding_column("product_reviews",module_spec="https://tfhub.dev/google/universal-sentence-encoder/2")

In [10]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/Wiki-words-500-with-normalization/2")
embeddings = embed(merged_df["reviewText"])


In [11]:
embeddings.shape

TensorShape([56351, 500])

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(embeddings)

array([[0.99999964, 0.9465526 , 0.7830856 , ..., 0.88230234, 0.8117728 ,
        0.7322055 ],
       [0.9465526 , 1.        , 0.75413275, ..., 0.83825684, 0.77073133,
        0.6778188 ],
       [0.7830856 , 0.75413275, 0.9999995 , ..., 0.8520808 , 0.7883779 ,
        0.6911081 ],
       ...,
       [0.88230234, 0.83825684, 0.8520808 , ..., 1.0000002 , 0.8910816 ,
        0.723804  ],
       [0.8117728 , 0.77073133, 0.7883779 , ..., 0.8910816 , 1.0000004 ,
        0.7359053 ],
       [0.7322055 , 0.6778188 , 0.6911081 , ..., 0.723804  , 0.7359053 ,
        1.0000005 ]], dtype=float32)

In [15]:
hub_layer = hub.KerasLayer("https://tfhub.dev/google/Wiki-words-500-with-normalization/2",
                           input_shape=[], dtype=tf.string)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 500)               504687500 
_________________________________________________________________
dense (Dense)                (None, 16)                8016      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 504,695,533
Trainable params: 8,033
Non-trainable params: 504,687,500
_________________________________________________________________
